<a href="https://colab.research.google.com/github/SSNascimento/Projeto_MCASP_Imersao/blob/main/Projeto_MCASP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai requests

In [ ]:
#Configurações iniciais
import google.generativeai as genai
from google.colab import userdata
import requests # Importe a biblioteca requests aqui

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [ ]:
#Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [ ]:
safety_settings={
    'HATE': 'BLOCK_SOME',
    'HARASSMENT': 'BLOCK_SOME',
    'SEXUAL' : 'BLOCK_SOME',
    'DANGEROUS' : 'BLOCK_SOME'
    }

Inicializando o Modelo.

In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

KeyError: 'block_some'

Carregando o URL do conteúdo do MCASP (Manual de Contabilidade Aplicada ao Setor Público)

In [ ]:
pdf_url = "https://drive.google.com/file/d/1Qri4FjwNZAWQjlW8_AKb63hyzm0a-wID/view?usp=sharing"


Baixando o PDF

In [ ]:
response = requests.get(pdf_url)
with open('MCASP_9a_Edicao.pdf', 'wb') as pdf_file:
    pdf_file.write(response.content)

Carregando o conteúdo do PDF MCASP

In [ ]:
with open('MCASP_9a_Edicao.pdf', 'rb') as pdf_file:
    pdf_content = pdf_file.read()

Função para fazer perguntas ao modelo sobre o MCASP

In [ ]:
def perguntar_mcasp(pergunta):
    response = model.generate_content(
        f"Com base no conteúdo deste documento PDF:\n\n{pdf_content}\n\n{pergunta}"
    )
    return response.text

Loop de perguntas e respostas

In [ ]:
chat = model.start_chat(history=[])

prompt = input('Esperando prompt: ')

while prompt != "fim":
    resposta = perguntar_mcasp(prompt)
    print("Resposta:", response.text, '\n\n')
    prompt = input('Esperando prompt: ')

Esperando prompt:  O que é Contabilidade Aplicada ao Setor Público?


ERROR:tornado.access:500 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 5562.05ms


InternalServerError: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

In [ ]:
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.0-pro',
        generation_config={'candidate_count': 1, 'temperature': 0.5},
        safety_settings={<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>},
        tools=None,
        system_instruction=None,
    ),
    history=[]
)

In [ ]:
chat.history

[]

In [ ]:
#Melhorando a visualização
#Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Imprimindo o histórico
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('-------------------------------------------')